The following code is responsible for converting MIDI files into text notation.
# I found a way to use the original python library!
Reference here: https://github.com/jameswenzel/mydy/blob/master/src/Containers.py
https://github.com/jameswenzel/Fractal-Midi/blob/master/script.py
https://github.com/vishnubob/python-midi

# Small test on opening MIDI file

In [12]:
from mydy import Events, FileIO, Containers, Constants
test=FileIO.read_midifile('file2.mid') #returns a Pattern with the MIDI file information (resolution ecc...), based on documentation https://github.com/jameswenzel/mydy/blob/master/src/FileIO.py

test.resolution=480 #qui sto settando quanti tick ho in una quarter note. Quindi ogni quarter note avra 480 ticks.
print(test) #seems that changing the BPM doesn't influence the ticks. I have to figure out how time is managed
#Resolution is the same as PPQ

mydy.Pattern(format=0, resolution=480, tracks=\
[mydy.Track(relative: True\
  [mydy.TrackNameEvent(tick=0.0, text='', data=[]),
   mydy.TimeSignatureEvent(tick=0.0, data=[4, 2, 36, 8]),
   mydy.TimeSignatureEvent(tick=0.0, data=[4, 2, 36, 8]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[60, 100]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[64, 100]),
   mydy.NoteOffEvent(tick=1920.0, channel=0, data=[60, 64]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[62, 100]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[67, 100]),
   mydy.NoteOffEvent(tick=1920.0, channel=0, data=[62, 64]),
   mydy.NoteOffEvent(tick=0.0, channel=0, data=[64, 64]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[64, 100]),
   mydy.NoteOffEvent(tick=0.0, channel=0, data=[67, 64]),
   mydy.NoteOffEvent(tick=1920.0, channel=0, data=[64, 64]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[67, 100]),
   mydy.NoteOnEvent(tick=1920.0, channel=0, data=[66, 100]),
   mydy.NoteOffEvent(tick=0.0, channel=0, data=[

In [13]:

track = test[0]
track_abs = track.make_ticks_abs()
#print(track_abs)

print(track_abs[3])

filtered_list=track_abs.filter(lambda e: isinstance(e, Events.NoteOnEvent))

print(filtered_list)

print(len(filtered_list))

print(filtered_list[2].data[1])
pattern=Containers.Pattern(fmt = 0)
print(test.resolution)
print(pattern.resolution)


mydy.NoteOnEvent(tick=0.0, channel=0, data=[60, 100])
mydy.Track(relative: False\
  [mydy.NoteOnEvent(tick=0.0, channel=0, data=[60, 100]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[64, 100]),
   mydy.NoteOnEvent(tick=1920.0, channel=0, data=[62, 100]),
   mydy.NoteOnEvent(tick=1920.0, channel=0, data=[67, 100]),
   mydy.NoteOnEvent(tick=3840.0, channel=0, data=[64, 100]),
   mydy.NoteOnEvent(tick=5760.0, channel=0, data=[67, 100]),
   mydy.NoteOnEvent(tick=7680.0, channel=0, data=[66, 100]),
   mydy.NoteOnEvent(tick=9600.0, channel=0, data=[67, 100]),
   mydy.NoteOnEvent(tick=11520.0, channel=0, data=[69, 100]),
   mydy.NoteOnEvent(tick=13440.0, channel=0, data=[71, 100]),
   mydy.NoteOnEvent(tick=15360.0, channel=0, data=[60, 100]),
   mydy.NoteOnEvent(tick=17280.0, channel=0, data=[62, 100]),
   mydy.NoteOnEvent(tick=19200.0, channel=0, data=[64, 100]),
   mydy.NoteOnEvent(tick=21120.0, channel=0, data=[67, 100]),
   mydy.NoteOnEvent(tick=23040.0, channel=0, data=[66, 100]),
  

In [30]:
notes = []
couple = ()

for element in filtered_list:
    
    print(element.tick)
    print(element.data[0])
    #storing couples of (tick_time,note_pitch)
    notes.append(tuple((element.data[0],element.tick)))

print('printing tuples')
print(notes)


0.0
60
0.0
64
1920.0
62
1920.0
67
3840.0
64
5760.0
67
7680.0
66
9600.0
67
11520.0
69
13440.0
71
15360.0
60
17280.0
62
19200.0
64
21120.0
67
23040.0
66
24960.0
67
26880.0
69
28800.0
71
printing tuples
[(60, 0.0), (64, 0.0), (62, 1920.0), (67, 1920.0), (64, 3840.0), (67, 5760.0), (66, 7680.0), (67, 9600.0), (69, 11520.0), (71, 13440.0), (60, 15360.0), (62, 17280.0), (64, 19200.0), (67, 21120.0), (66, 23040.0), (67, 24960.0), (69, 26880.0), (71, 28800.0)]


In [16]:
test= Constants.C_3
print(test)

36


In [42]:
from mydy import Events, FileIO, Containers
import pdb

#Function responsible for converting midi notes into text. Since i have to train my network over the structure i decided
#which is 0b0000000 for no note, 0b01000000 for kick ecc... i need to convert midi notes into this format.

#The original script used for midi-text translation has been lost, must be re-implemented again
PPQ = 480 # Pulse per quater note. Used in sequencers. Standard value
event_per_bar = 16 # to quantise.
min_ppq = PPQ / (event_per_bar/4)

# ignore: 39 hand clap, 54 tambourine, 56 Cowbell, 58 Vibraslap, 60-81

#the dictionary below maps values to other ones. Reduced the size of the used notes. For example
#if i have an eletric snare or a stick snare, i just map both of them into a standard snare

drum_conversion = {35:36, # acoustic bass drum -> bass drum (36)
					37:38, 40:38, # 37:side stick, 38: acou snare, 40: electric snare
					43:41, # 41 low floor tom, 43 ghigh floor tom
					47:45, # 45 low tom, 47 low-mid tom
					50:48, # 50 high tom, 48 hi mid tom
					44:42, # 42 closed HH, 44 pedal HH
					57:49, # 57 Crash 2, 49 Crash 1
					59:51, 53:51, 55:51, # 59 Ride 2, 51 Ride 1, 53 Ride bell, 55 Splash
					52:49 # 52: China cymbal
					}

#Used in the code to map elements, everything that has not one of the following number is discarded.
#Basically i'm ignoring notes that are not in my dataset (for examle i'll ignore shakers ecc...)
				# k, sn,cHH,oHH,LFtom,ltm,htm,Rde,Crash
allowed_pitch = [36, 38, 42, 46, 41, 45, 48, 51, 49] # 46: open HH
cymbals_pitch = [49, 51] # crash, ride
cymbals_pitch = [] # crash, ride
# pitch_to_midipitch = {36:midi.C_2, # kick # for general MIDI Drum map
# 						38:midi.D_2, # Snare
# 						39:midi.Eb_2, # hand clap (it's alive by mistake..)
# 						41:midi.F_2, # Low floor tom
# 						42:midi.Gb_2, # Close HH
# 						45:midi.A_2, # Low tom
# 						46:midi.Bb_2, # Open HH
# 						48:midi.C_3,  # Hi Mid Tom
# 						49:midi.Db_3, # Crash
# 						51:midi.Eb_3 # Ride
# 						}

#mapping midi values into notes
pitch_to_midipitch = {36:Constants.C_3,  # for logic 'SoCal' drum mapping
						38:Constants.D_3, 
						39:Constants.Eb_3,
						41:Constants.F_3,
						42:Constants.Gb_3,
						45:Constants.A_3,
						46:Constants.Bb_3,
						48:Constants.C_4,
						49:Constants.Db_4,
						51:Constants.Eb_4
						}
#la singola nota è un elemento composto da pitch (numerico, pitch midi) e tick (modo per tenere il tempo in midi)
class Note:
	def __init__(self, pitch, c_tick):
		self.pitch = pitch
		self.c_tick = c_tick # cumulated_tick of a midi note

	def add_index(self, idx):
		'''index --> 16-th note-based index starts from 0'''
		self.idx = idx

class Note_List():
	def __init__(self):
		''''''
		self.notes = []
		self.quantised = False
		self.max_idx = None

	def add_note(self, note):
		'''note: instance of Note class'''
		self.notes.append(note)

	def quantise(self, minimum_ppq):
		'''
		e.g. if minimum_ppq=120, quantise by 16-th note.
		
		'''
		if not self.quantised:
			for note in self.notes:
				note.c_tick = ((note.c_tick+minimum_ppq/2)//minimum_ppq)* minimum_ppq # quantise
				#here the index is calculated. The index is an absolute index over the 16th notes.
				#for example an index of value 34, means that my current note appears after 34 chromes
				#it's simply calculated by dividing the cumulated tick of the note by the ticks contained in a 16th note
				note.add_index(note.c_tick/minimum_ppq)
            #NB: THE QUANTIZATION FUNCTION ITERATES OVER ALL THE NOTES. So first i add all the notes, then i iterate and quantize

			#Does this automatically reference to the last item added?
			#YES. The counter note will store the last element of the iteration. So basically here i'm assigning as max index the index of the last added note
			self.max_idx = note.idx

			#Here checks if if my ending is a full musical bar. For example, if my file ends with a single kick, i'll add that note.
			#but that kick will (probably) be at the beginning of the last musical bar. So i have to "pad" until the end.
			#It's like adding a pause on my piece, so i have all complete bars and no trucated ones at the end
			if (self.max_idx + 1) % event_per_bar != 0:
				self.max_idx += event_per_bar - ((self.max_idx + 1) % event_per_bar) # make sure it has a FULL bar at the end.
			self.quantised = True

		return

	def simplify_drums(self):
		''' use only allowed pitch - and converted not allowed pitch to the similar in a sense of drums!
		'''
        #Here forces conversion into the pitches in drum_conversion
		for note in self.notes:
			if note.pitch in drum_conversion: # ignore those not included in the key
				note.pitch = drum_conversion[note.pitch]
		#https://stackoverflow.com/questions/30670310/what-do-brackets-in-a-for-loop-in-python-mean
		#The following one is a list comprehension. Basically generates a new list from an existing one using a given condition on the elements
		self.notes = [note for note in self.notes if note.pitch in allowed_pitch]	
				
		return
	
	def return_as_text(self):
		''''''
		length = self.max_idx + 1 # of events in the track.
		event_track = []
		#Thw following cycle create a 9 by N matrix. I append N times a vector of nine zeros.
		#This means that i create N notes, and then i initialize them with all zeros (9 zeros, since a note is represented by a 9 element binary number)

		for note_idx in range(length):  #sostituire xrange con range in Python3
			event_track.append(['0']*len(allowed_pitch))
			
		num_bars = length/event_per_bar# + ceil(len(event_texts_temp) % _event_per_bar)

		for note in self.notes:
			pitch_here = note.pitch
			#The following line returns the index of the passed pitch. Basically given an input generic pitch
			#it returns the associated pitch in my vocabolary (computes the actual mapping from the whole
			#vocabolary of notes into my reduced one)
			note_add_pitch_index = allowed_pitch.index(pitch_here) # 0-8
			event_track[note.idx][note_add_pitch_index] = '1'
			# print note.idx, note.c_tick, note_add_pitch_index, ''.join(event_track[note.idx])
			# pdb.set_trace()
			
		event_text_temp = ['0b'+''.join(e) for e in event_track] # encoding to binary
		
		event_text = []
		# event_text.append('SONG_BEGIN')
		# event_text.append('BAR')
		for bar_idx in range(num_bars):
			event_from = bar_idx * event_per_bar
			event_to = event_from + event_per_bar
			event_text = event_text + event_text_temp[event_from:event_to]
			event_text.append('BAR')

		# event_text.append('SONG_END')

		return ' '.join(event_text)
		

In [45]:

##NB: AGGIUNGERE GLI idx ALLE SINGLE NOTES! 
note_list = Note_List()
for note in notes:
    pitch = note[0]
    tick = note[1]
    new_note = Note(pitch,tick)
    note_list.add_note(new_note)


In [46]:
print('printing ticks before quantization')
for note in note_list.notes:
    print(note.c_tick)
    
note_list.quantise(min_ppq)

print('printing ticks after quantization')
for note in note_list.notes:
    print(note.c_tick)

printing ticks before quantization
0.0
0.0
1920.0
1920.0
3840.0
5760.0
7680.0
9600.0
11520.0
13440.0
15360.0
17280.0
19200.0
21120.0
23040.0
24960.0
26880.0
28800.0
printing ticks after quantization
0.0
0.0
1920.0
1920.0
3840.0
5760.0
7680.0
9600.0
11520.0
13440.0
15360.0
17280.0
19200.0
21120.0
23040.0
24960.0
26880.0
28800.0


In [ ]:
note_list.